In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mswarbrickjones/reddit-selfposts")


print("Path to dataset files:", path)

100%|██████████| 352M/352M [00:03<00:00, 102MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mswarbrickjones/reddit-selfposts/versions/3


In [ ]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm


In [ ]:
subreddit_info = pd.read_csv("/root/.cache/kagglehub/datasets/mswarbrickjones/reddit-selfposts/versions/3/subreddit_info.csv")
data = pd.read_csv("/root/.cache/kagglehub/datasets/mswarbrickjones/reddit-selfposts/versions/3/rspct.tsv", delimiter='\t')

In [ ]:
small_data = data[data['subreddit'].isin(["talesfromtechsupport", "teenmom", "Harley", "ringdoorbell", "intel", "residentevil", "BATProject", "hockeyplayers", "asmr", \
             "rawdenim", "steinsgate", "DBZDokkanBattle", "Nootropics", "l5r", "NameThatSong", "homeless", "antidepressants", "absolver", "Warframe", "fatpeoplestories"])]
small_data.head()

,id,subreddit,title,selftext
0,6d8knd,talesfromtechsupport,Remember your command line switches...,"Hi there, <lb>The usual. Long time lerker, fi..."
1,58mbft,teenmom,"So what was Matt ""addicted"" to?",Did he ever say what his addiction was or is h...
2,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...
3,6ti6re,ringdoorbell,"Not door bell, but floodlight mount height.",I know this is a sub for the 'Ring Doorbell' b...
4,77sxto,intel,Worried about my 8700k small fft/data stress r...,"Prime95 (regardless of version) and OCCT both,..."


In [ ]:
small_data.to_csv("small_data.csv")

In [ ]:
subreddit_info = subreddit_info[subreddit_info['in_data'] == True]
subreddit_info.head()

,subreddit,category_1,category_2,category_3,in_data,reason_for_exclusion
0,whatsthatbook,advice/question,book,NaN,True,NaN
25,theydidthemath,advice/question,calculations,NaN,True,NaN
26,datarecovery,advice/question,data recovery,NaN,True,NaN
27,declutter,advice/question,declutter,NaN,True,NaN
30,productivity,advice/question,discipline,NaN,True,NaN


In [ ]:
def fill_embeddings(df):
  embeds = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  copy_df = df.copy()
  embeds = embeds.encode(df['selftext'].tolist())
  copy_df['selftext_minilm_embeddings'] = [embed for embed in embeds]
  return copy_df



In [ ]:
data_with_encodings = fill_embeddings(small_data.sample(n=500))
data_with_encodings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,id,subreddit,title,selftext,selftext_minilm_embeddings
672828,66sazr,hockeyplayers,Pacific Rink Player Bag Pictures,I am not one to usually drop big money on gear...,"[-0.044816352, 0.10744359, -0.009349173, -0.00..."
278536,6txuah,hockeyplayers,Help,"Hey, I'm a 18 year old originally from the UK,...","[0.08992301, -0.021913763, 0.023310777, -0.009..."
116664,4x00v8,homeless,First night in my car,"I'm not sure why I'm even posting, I've never ...","[-0.0069108154, -0.088503264, 0.02725935, 0.03..."
626181,8fgdps,antidepressants,Do ssris change your appearance?,I've been on ssris since i was 16 (20 now). I'...,"[0.008210345, 0.03255544, -0.062979, 0.0528693..."
907392,7eo13g,DBZDokkanBattle,Global and Jap on iOS help!!,Hey guys so I’ve been looking around for an an...,"[-0.066853404, -0.111486256, 0.023732524, -0.1..."
...,...,...,...,...,...
993450,576sgl,teenmom,"What ever happened to Gary's hot friend, Evan?","The first few seasons of TM, whenever they sho...","[-0.138958, -0.046597984, -0.031697206, -0.032..."
214990,88gzu0,rawdenim,Updated Railcar Journeyman update. (they are h...,Hello /r/rawdenim <lb><lb>As some of you know ...,"[-0.07894018, 0.03604362, 0.0063781096, 0.0325..."
508994,7p1dpf,hockeyplayers,Hockey scene in Reno?,Might be relocating to Reno and the first thin...,"[0.08834141, -0.07823963, 0.028159013, 0.07313..."
912391,76210w,l5r,The Wandering Ronin is a really stylish card.,I know that many see the card as wasteful. Mos...,"[-0.078984894, 0.029264877, -0.059397228, -0.0..."


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
def logistic_regression_benchmark(df):
  X = np.array(df['selftext_minilm_embeddings'])
  # X.shape = (100,)
  # But the shape is known to be (100, 348)
  # Force it to be this way
  X = np.array([np.array(x) for x in X])
  y = df['subreddit']

  le = LabelEncoder()
  y_encoded = le.fit_transform(y)

  X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
  # Create a logistic regression model
  model = LogisticRegression()

  # Train the model
  model.fit(X_train, y_train)

  # Make predictions on the test set
  y_pred = model.predict(X_test)

  # Evaluate the model
  accuracy = accuracy_score(y_test, y_pred)
  print(f'Accuracy: {accuracy}')

  # Print classification report
  print(classification_report(y_test, y_pred, target_names=le.classes_))

In [ ]:
logistic_regression_benchmark(data_with_encodings)

Accuracy: 0.82
                      precision    recall  f1-score   support

          BATProject       1.00      0.33      0.50         3
     DBZDokkanBattle       1.00      0.80      0.89         5
              Harley       1.00      1.00      1.00         3
        NameThatSong       0.60      1.00      0.75         3
          Nootropics       1.00      1.00      1.00         1
            Warframe       0.64      1.00      0.78         7
            absolver       1.00      0.62      0.77         8
     antidepressants       1.00      1.00      1.00         6
                asmr       0.83      0.83      0.83         6
    fatpeoplestories       1.00      1.00      1.00         5
       hockeyplayers       0.80      1.00      0.89         4
            homeless       1.00      1.00      1.00         6
               intel       1.00      0.75      0.86         4
                 l5r       1.00      0.33      0.50         3
            rawdenim       1.00      0.67      0.80   

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [ ]:
def logistic_regression_benchmark_bow(df):
  x = df['selftext']
  y = df['subreddit']

  le = LabelEncoder()
  y_encoded = le.fit_transform(y)
  X_train, X_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

  vectorizer = CountVectorizer()
  X_train_bow = vectorizer.fit_transform(X_train)
  X_test_bow = vectorizer.transform(X_test)
  model = LogisticRegression()
  model.fit(X_train_bow, y_train)
  y_pred = model.predict(X_test_bow)

  accuracy = accuracy_score(y_test, y_pred)
  print(f'Accuracy: {accuracy}')

  # Print classification report
  print(classification_report(y_test, y_pred, target_names=le.classes_))


In [ ]:
logistic_regression_benchmark_bow(small_data)

Accuracy: 0.93625
                      precision    recall  f1-score   support

          BATProject       0.97      0.94      0.95       192
     DBZDokkanBattle       0.94      0.96      0.95       206
              Harley       0.94      0.94      0.94       219
        NameThatSong       0.97      0.98      0.97       188
          Nootropics       0.86      0.89      0.88       212
            Warframe       0.88      0.92      0.90       197
            absolver       0.88      0.89      0.88       192
     antidepressants       0.91      0.90      0.91       210
                asmr       0.95      0.96      0.95       187
    fatpeoplestories       0.92      0.94      0.93       195
       hockeyplayers       0.95      0.94      0.95       197
            homeless       0.95      0.92      0.94       196
               intel       0.96      0.97      0.97       191
                 l5r       0.96      0.98      0.97       205
            rawdenim       0.97      0.93      0.95

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
def logistic_regression_benchmark_tfidf(df):
  x = df['selftext']
  y = df['subreddit']

  le = LabelEncoder()
  y_encoded = le.fit_transform(y)
  X_train, X_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)
  vectorizer = TfidfVectorizer()
  X_train_tfidf = vectorizer.fit_transform(X_train)
  X_test_tfidf = vectorizer.transform(X_test)
  model = LogisticRegression()
  model.fit(X_train_tfidf, y_train)
  y_pred = model.predict(X_test_tfidf)

  accuracy = accuracy_score(y_test, y_pred)
  print(f'Accuracy: {accuracy}')

  # Print classification report
  print(classification_report(y_test, y_pred, target_names=le.classes_))

In [ ]:
logistic_regression_benchmark_tfidf(small_data)

Accuracy: 0.94875
                      precision    recall  f1-score   support

          BATProject       0.99      0.94      0.97       192
     DBZDokkanBattle       0.97      0.95      0.96       206
              Harley       0.95      0.95      0.95       219
        NameThatSong       0.97      0.97      0.97       188
          Nootropics       0.90      0.93      0.92       212
            Warframe       0.89      0.94      0.91       197
            absolver       0.94      0.93      0.93       192
     antidepressants       0.93      0.95      0.94       210
                asmr       0.98      0.96      0.97       187
    fatpeoplestories       0.90      0.94      0.92       195
       hockeyplayers       0.98      0.96      0.97       197
            homeless       0.95      0.95      0.95       196
               intel       0.99      0.98      0.99       191
                 l5r       0.96      0.95      0.95       205
            rawdenim       0.96      0.94      0.95

In [ ]:
# Preprocessing
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    # Lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^a-z\s]', '', text)

    # Remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

def split_data(percent, data):
    X = data['selftext']
    Y = data['subreddit']
    return train_test_split(
      X, Y, test_size=percent, random_state=42, stratify=Y
    )

def preprocess_pipeline(data):
    # Drop null values
    data_clean = data.dropna(subset=['selftext', 'subreddit'])

    # Clean posts
    data_clean['selftext'] = [preprocess(p) for p in data_clean['selftext']]

    # Split data
    X_train, X_test, y_train, y_test = split_data(0.2, data_clean)

    return X_train, X_test, y_train, y_test

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# SVM
def svm(data):
  # Preprocess data
  X_train, X_test, y_train, y_test = preprocess_pipeline(data)

  vectorizer = TfidfVectorizer(
    max_features=10000,
    stop_words='english',
    lowercase=True,
    ngram_range=(1,2)
  )

  vectorizer_bow = CountVectorizer(
    max_features=10000,
    stop_words='english',
    lowercase=True,
    ngram_range=(1,2)
  )

  X_train_tfidf = vectorizer.fit_transform(X_train)
  X_test_tfidf = vectorizer.transform(X_test)

  X_train_bow = vectorizer_bow.fit_transform(X_train)
  X_test_bow = vectorizer_bow.transform(X_test)

  clf = LinearSVC()
  clf.fit(X_train_tfidf, y_train)

  y_pred = clf.predict(X_test_tfidf)
  print(classification_report(y_test, y_pred))

  clf_bow = LinearSVC()
  clf_bow.fit(X_train_bow, y_train)

  y_pred_bow = clf_bow.predict(X_test_bow)
  print(classification_report(y_test, y_pred_bow))

svm(small_data)

                      precision    recall  f1-score   support

          BATProject       1.00      0.97      0.99       200
     DBZDokkanBattle       0.95      0.94      0.95       200
              Harley       0.97      0.92      0.95       200
        NameThatSong       0.98      0.98      0.98       200
          Nootropics       0.90      0.92      0.91       200
            Warframe       0.88      0.90      0.89       200
            absolver       0.92      0.92      0.92       200
     antidepressants       0.92      0.94      0.93       200
                asmr       0.98      0.97      0.98       200
    fatpeoplestories       0.95      0.96      0.95       200
       hockeyplayers       0.98      0.94      0.96       200
            homeless       0.96      0.96      0.96       200
               intel       0.95      0.95      0.95       200
                 l5r       0.95      0.94      0.94       200
            rawdenim       0.96      0.97      0.97       200
       

In [ ]:
def svm_emb(df):
  X = np.array(df['selftext_minilm_embeddings'])

  X = np.array([np.array(x) for x in X])
  y = df['subreddit']

  le = LabelEncoder()
  y_encoded = le.fit_transform(y)

  X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
  model = LinearSVC()

  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(f'Accuracy: {accuracy}')

  print(classification_report(y_test, y_pred, target_names=le.classes_))

In [ ]:
svm_emb(data_with_encodings)

Accuracy: 0.85
                      precision    recall  f1-score   support

          BATProject       0.80      1.00      0.89         4
     DBZDokkanBattle       1.00      1.00      1.00         4
              Harley       0.67      1.00      0.80         2
        NameThatSong       0.67      1.00      0.80         2
          Nootropics       1.00      0.83      0.91         6
            Warframe       1.00      0.17      0.29         6
            absolver       0.67      1.00      0.80         4
     antidepressants       0.80      1.00      0.89         4
                asmr       1.00      1.00      1.00         4
    fatpeoplestories       0.75      1.00      0.86         3
       hockeyplayers       0.88      0.78      0.82         9
            homeless       1.00      1.00      1.00         6
               intel       1.00      1.00      1.00         4
                 l5r       0.83      0.83      0.83         6
            rawdenim       0.67      0.67      0.67   

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
# MLP with embeddings
def MLP_emb(df):
  df_emb = fill_embeddings(df)
  X_train, X_test, y_train, y_test = train_test_split(df_emb['selftext_minilm_embeddings'], df_emb['subreddit'], test_size=0.2, random_state=42)

  X_train_emb = df_emb.loc[X_train.index, 'selftext_minilm_embeddings']
  X_test_emb = df_emb.loc[X_test.index, 'selftext_minilm_embeddings']

  train_emb = np.vstack(X_train_emb.values)
  test_emb = np.vstack(X_test_emb.values)

  model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
  model.fit(train_emb, y_train)

  y_pred = model.predict(test_emb)
  print(classification_report(y_test, y_pred))


In [ ]:
MLP_emb(small_data)

                      precision    recall  f1-score   support

          BATProject       0.99      0.96      0.98       192
     DBZDokkanBattle       0.96      0.93      0.95       206
              Harley       0.97      0.95      0.96       219
        NameThatSong       0.98      0.99      0.99       188
          Nootropics       0.94      0.94      0.94       212
            Warframe       0.87      0.85      0.86       197
            absolver       0.88      0.93      0.91       192
     antidepressants       0.94      0.97      0.96       210
                asmr       0.98      0.95      0.96       187
    fatpeoplestories       0.94      0.93      0.94       195
       hockeyplayers       0.96      0.96      0.96       197
            homeless       0.96      0.97      0.96       196
               intel       0.98      0.98      0.98       191
                 l5r       0.91      0.88      0.90       205
            rawdenim       0.97      0.97      0.97       207
       

In [ ]:
# MLP with bag-of-words
def MLP_bow(df):
  X_train, X_test, y_train, y_test = preprocess_pipeline(df)
  vectorizer = CountVectorizer()
  x_train_bow = vectorizer.fit_transform(X_train)
  x_test_bow = vectorizer.transform(X_test)

  model = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=20, random_state=42)
  model.fit(x_train_bow, y_train)

  y_pred = model.predict(x_test_bow)
  print(classification_report(y_test, y_pred))

In [ ]:
MLP_bow(small_data)

                      precision    recall  f1-score   support

          BATProject       0.99      0.99      0.99       200
     DBZDokkanBattle       0.94      0.94      0.94       200
              Harley       0.98      0.94      0.96       200
        NameThatSong       0.98      0.98      0.98       200
          Nootropics       0.97      0.89      0.93       200
            Warframe       0.91      0.88      0.89       200
            absolver       0.89      0.93      0.90       200
     antidepressants       0.90      0.97      0.93       200
                asmr       0.97      0.98      0.97       200
    fatpeoplestories       0.99      0.94      0.96       200
       hockeyplayers       0.94      0.94      0.94       200
            homeless       0.92      0.98      0.95       200
               intel       0.94      0.98      0.96       200
                 l5r       0.95      0.94      0.95       200
            rawdenim       0.98      0.98      0.98       200
       

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# MLP with TF-IDF
def MLP_tfidf(df):
  X_train, X_test, y_train, y_test = preprocess_pipeline(df)
  vectorizer = TfidfVectorizer()
  x_train_tfidf = vectorizer.fit_transform(X_train)
  x_test_tfidf = vectorizer.transform(X_test)

  model= MLPClassifier(hidden_layer_sizes=(100,50), max_iter=20, random_state=42)
  model.fit(x_train_tfidf, y_train)

  y_pred = model.predict(x_test_tfidf)
  print(classification_report(y_test, y_pred))

In [ ]:
MLP_tfidf(small_data)

                      precision    recall  f1-score   support

          BATProject       0.98      0.99      0.99       200
     DBZDokkanBattle       0.97      0.94      0.95       200
              Harley       0.99      0.94      0.96       200
        NameThatSong       0.96      0.98      0.97       200
          Nootropics       0.96      0.91      0.93       200
            Warframe       0.90      0.90      0.90       200
            absolver       0.93      0.91      0.92       200
     antidepressants       0.92      0.95      0.94       200
                asmr       0.97      0.97      0.97       200
    fatpeoplestories       0.96      0.97      0.96       200
       hockeyplayers       0.96      0.96      0.96       200
            homeless       0.92      0.97      0.95       200
               intel       0.95      0.97      0.96       200
                 l5r       0.95      0.95      0.95       200
            rawdenim       0.99      0.98      0.98       200
       

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
